In [66]:
import re
import pickle
import string
import matplotlib.pyplot as plt
from collections import Counter

In [6]:
with open('./text/final_text.bin', 'rb') as f:
    final_text= pickle.load(f)

In [39]:
def preprocessing(text):
    result= ''
    for ch in text:
        if ch in string.punctuation:
            result += ' '
        else:
            result += ch
    result = ' '.join(result.split())  # '\n'을 제거하기 위해 split() 후 다시 합침
    result= re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣]',' ',result)
    while '  ' in result:
        result= re.sub('  ',' ', result)
    return result

In [42]:
cleansed_text= preprocessing(final_text)

In [51]:
def return_morph(text):
    import MeCab
    from konlpy.tag import Mecab
    mecab= Mecab(dicpath= 'C:\\mecab\\mecab-ko-dic')
    result= mecab.morphs(text)
    result= [res for res in result if len(res) >= 2]
    return result

In [53]:
morphs= return_morph(cleansed_text)

In [65]:
Counter(morphs).most_common(50)

[('환경', 7479),
 ('으로', 5165),
 ('관리', 4339),
 ('에서', 2597),
 ('지역', 2262),
 ('사업', 2156),
 ('시설', 1931),
 ('계획', 1914),
 ('오염', 1830),
 ('추진', 1763),
 ('물질', 1719),
 ('위해', 1653),
 ('정책', 1652),
 ('배출', 1640),
 ('폐기물', 1624),
 ('기준', 1509),
 ('기술', 1503),
 ('위한', 1435),
 ('대한', 1385),
 ('개발', 1335),
 ('운영', 1314),
 ('생물', 1309),
 ('자원', 1296),
 ('생태', 1292),
 ('활용', 1284),
 ('조사', 1240),
 ('부터', 1223),
 ('제도', 1221),
 ('수질', 1213),
 ('지원', 1212),
 ('지속', 1190),
 ('국가', 1184),
 ('해양', 1164),
 ('처리', 1145),
 ('산업', 1142),
 ('강화', 1121),
 ('환경부', 1094),
 ('시행', 1084),
 ('정보', 1083),
 ('보전', 1062),
 ('대상', 1025),
 ('가능', 1020),
 ('개선', 1014),
 ('평가', 1009),
 ('관한', 971),
 ('개정', 947),
 ('분야', 943),
 ('화학', 934),
 ('현황', 922),
 ('제품', 909)]